# Name :Awais Amajd
# Reg. No. :2022cs541

# Symmetric Encryption Lab

In this lab, you will be writing your own cryptographic library to decrypt a substitution cipher, and using existing cryptographic libraries to experiment with a symmetric encryption called AES and a classic encryption named Caesar cipher.

## 1.1 Substitution Cipher (5 points)

### Files
1. **sub_key.txt**: key  
2. **sub_ciphertext.txt**: ciphertext  

`sub_key.txt` contains a permutation of the 26 upper-case letters that represents the key for a substitution cipher. Using this key, the ith letter in the alphabet in the plaintext has been replaced by the ith letter in `sub_key.txt` to produce ciphertext in `sub_ciphertext.txt`. 

For example, if the first three letters in your `sub_key.txt` are `ZDF...`, then:
- All **As** in the plaintext have become **Zs** in the ciphertext
- All **Bs** have become **Ds**
- All **Cs** have become **Fs**.

The plaintext we encrypted contains only upper-case letters, numbers, and spaces. Numbers and spaces in the plaintext were not encrypted. They appear exactly as they did in the plaintext.

### Task
Submit the plaintext, which is obtained using the key in `sub_key.txt` to decrypt `sub_ciphertext.txt`, in the file **solution01.txt**.

### Cryptographic Library
For the following tasks, we recommend using **PyCrypto**, an open-source crypto library for Python. 

PyCrypto can be installed using:
```bash
 pip install pycrypto


In [5]:
# Step 1: Read the key and ciphertext from the files
with open('./substitution/sub_key.txt', 'r') as key_file:
    substitution_key = key_file.read().strip()

with open('./substitution/sub_ciphertext.txt', 'r') as cipher_file:
    ciphertext = cipher_file.read().strip()

# Step 2: Create a decryption map
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
decryption_map = {substitution_key[i]: alphabet[i] for i in range(26)}

# Step 3: Decrypt the ciphertext
plaintext = []
for char in ciphertext:
    if char in decryption_map:
        plaintext.append(decryption_map[char])
    else:
        plaintext.append(char)  # Leave numbers and spaces unchanged

# Join the plaintext list into a string
plaintext = ''.join(plaintext)

# Step 4: Write the decrypted plaintext to solution01.txt
with open('./substitution/solution01.txt', 'w') as output_file:
    output_file.write(plaintext)

print("Decryption complete. Plaintext written to solution01.txt.")


Decryption complete. Plaintext written to solution01.txt.


# 1.2 AES Encryption (5 points)

### Files
1. **aes_key.hex**: key  
2. **aes_iv.hex**: initialization vector  
3. **aes_ciphertext.hex**: ciphertext  

`aes_key.hex` contains a 256-bit AES key represented as an ASCII string of hexadecimal values. `aes_iv.hex` contains a 128-bit Initialization Vector (IV) in a similar representation. We encrypted a sentence using **AES in CBC mode** with this key and IV, and wrote the resulting ciphertext (also stored in hexadecimal) in `aes_ciphertext.hex`.

### Task
Decrypt the ciphertext using the provided information and submit the plaintext in **solution02.txt**.


In [9]:
from Crypto.Cipher import AES
import binascii

# Step 1: Read the key, IV, and ciphertext from the hex files
with open('./Aes_Encryption/aes_key.hex', 'r') as key_file:
    key_hex = key_file.read().strip()

with open('./Aes_Encryption/aes_iv.hex', 'r') as iv_file:
    iv_hex = iv_file.read().strip()

with open('./Aes_Encryption/aes_ciphertext.hex', 'r') as ciphertext_file:
    ciphertext_hex = ciphertext_file.read().strip()

# Step 2: Convert hex strings to bytes
key = binascii.unhexlify(key_hex)
iv = binascii.unhexlify(iv_hex)
ciphertext = binascii.unhexlify(ciphertext_hex)

# Step 3: Create AES cipher object in CBC mode
cipher = AES.new(key, AES.MODE_CBC, iv)

# Step 4: Decrypt the ciphertext
plaintext = cipher.decrypt(ciphertext)

# Step 5: Remove padding (PKCS7 padding is often used with AES in CBC mode)
padding_length = plaintext[-1]
plaintext = plaintext[:-padding_length]

# Step 6: Write the decrypted plaintext to solution02.txt
with open('./Aes_Encryption/solution02.txt', 'wb') as solution_file:
    solution_file.write(plaintext)

print("Decryption complete. Plaintext written to solution02.txt.")


Decryption complete. Plaintext written to solution02.txt.


# 1.3 AES: Breaking A Weak AES Key (5 points)

### Files
1. **aes_weak_ciphertext.hex**: ciphertext  

As with the last task, we encrypted a sentence using **256-bit AES in CBC mode** and stored the result in hexadecimal in the file `aes_weak_ciphertext.hex`. For this task, however, we haven’t supplied the key. 

All we’ll tell you about the key is that:
- It is 256 bits long.
- Its **251 most significant (leftmost) bits** are all `0`s.
- The initialization vector (IV) was set to all `0`s.

### Task
First, find all possible plaintexts in the given key space. Then, you will review the plaintexts to find the correct one along with its corresponding key.

Submit the **key** of the appropriate plaintext as a hexadecimal string in **Solution03.hex**.


In [7]:
import binascii
from Crypto.Cipher import AES
import re

# Function to check if the text is English-like
def is_english_like(text):
    # Simple heuristic to check for English-like text
    # A real implementation would be more robust
    if not text:
        return False
    # Check if the text contains a reasonable number of spaces and common English letters
    return bool(re.search(r'[A-Z ]', text))

def decrypt_and_check(ciphertext_hex, key):
    # Create AES cipher in CBC mode with a zeroed IV
    cipher = AES.new(key, AES.MODE_CBC, iv=b"\x00" * 16)
    # Decrypt the ciphertext
    plaintext_bytes = cipher.decrypt(binascii.unhexlify(ciphertext_hex))
    
    # Try to remove padding (PKCS7 padding)
    padding_len = plaintext_bytes[-1]
    if 0 < padding_len <= 16:  # AES block size is 16 bytes
        plaintext_bytes = plaintext_bytes[:-padding_len]
    
    try:
        # Decode the plaintext bytes to a string
        return plaintext_bytes.decode("utf-8")
    except UnicodeDecodeError:
        # Return empty string if decoding fails
        return ""

def break_weak_aes(ciphertext_hex):
    # Generate keys with first 251 bits as zeroes and varying the last byte
    key_space = [b"\x00" * 31 + bytes([i]) for i in range(32)]
    
    for key in key_space:
        plaintext = decrypt_and_check(ciphertext_hex, key)
        
        # Check if the decrypted plaintext resembles English text
        if is_english_like(plaintext):
            return plaintext, key  # Return both plaintext and key

    return None, None  # If no valid plaintext found

# Given ciphertext
ciphertext_hex = "3f45b1f996d8e30dc9885ea7f452b5b27682f345e389a04fb4e965e145d06bd711bc2463a00a8b20234e2b8acb200d6298b7d8b8ca7f3b2f8fdc2d5a9e8c0a9fe377ae1c3310fdacc37b0b8e8c692c50c6a8833d15e492a1a67247e785421631"

# Attempt to break the weak AES encryption
plaintext, key = break_weak_aes(ciphertext_hex)

if plaintext:
    print("Plaintext:", plaintext)
    # Save the plaintext to a solution file
    with open("./aes_breaking/solution.txt", "w") as solution_file:
        solution_file.write(f"Plaintext: {plaintext}\n")
        solution_file.write(f"Key (in hex): {binascii.hexlify(key).decode()}\n")
else:
    print("No valid plaintext found.")


Plaintext: EDMUND RANDOLPH HELPED DRAFT  RATIFY THE CONSTITUTION BEFORE BECOMING THIS MANS ATTORNEY GENERAL


# Breaking Caesar Cipher

## Task Description
Write a program in Python that will decipher a Caesar cipher without knowing the key. Your program must be able to find the key and the correct plaintext without any human intervention.

## Requirements
- Implement a brute-force method to try all possible shifts (1 to 25).
- Analyze the decrypted outputs to determine the most likely plaintext.
- The program should automatically identify the key and the plaintext.

## Example Implementation
Below is an example Python program that accomplishes this task:

In [10]:
import string

def caesar_decrypt(ciphertext, shift):
    decrypted_text = []
    
    for char in ciphertext:
        if char in string.ascii_uppercase:
            # Shift the character and wrap around if necessary
            new_char = chr((ord(char) - 65 - shift) % 26 + 65)
            decrypted_text.append(new_char)
        else:
            # If not a letter, keep it as is (numbers, spaces, punctuation)
            decrypted_text.append(char)

    return ''.join(decrypted_text)

def is_english_like(text):
    # Check if the text contains a reasonable number of spaces and common English letters
    # This is a simple heuristic; a more robust implementation would be better
    words = text.split()
    num_words = len(words)
    if num_words == 0:
        return False
    num_valid_words = sum(1 for word in words if word.isalpha())
    return num_valid_words / num_words > 0.5  # At least 50% valid words

def break_caesar(ciphertext):
    for shift in range(1, 26):  # Try all possible shifts (1 to 25)
        decrypted_text = caesar_decrypt(ciphertext, shift)
        if is_english_like(decrypted_text):
            return decrypted_text, shift  # Return the likely plaintext and the key

    return None, None  # If no valid plaintext found

# Example ciphertext
ciphertext = "GUR DHVP XLBH NAR XLBH ZL GUR CNEX."  # Example Caesar cipher

# Attempt to break the Caesar cipher
plaintext, key = break_caesar(ciphertext)

if plaintext:
    print("Plaintext:", plaintext)
    print("Key:", key)
else:
    print("No valid plaintext found.")


Plaintext: FTQ CGUO WKAG MZQ WKAG YK FTQ BMDW.
Key: 1
